In [19]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    ! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py
    import setup_google_colab
    setup_google_colab.setup_project()

import sys
sys.path.append("..")
from common.download_utils import download_project_resources

download_project_resources()

--2020-08-07 14:28:29--  https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1939 (1.9K) [text/plain]
Saving to: ‘setup_google_colab.py’

setup_google_colab. 100%[===================>]   1.89K  --.-KB/s    in 0s      

2020-08-07 14:28:29 (27.3 MB/s) - ‘setup_google_colab.py’ saved [1939/1939]



In [1]:
import numpy as np
import pandas as pd
import csv
import pickle
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

from dialogue_manager import DialogueManager
from utils import *
from utils import text_prepare

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Intent and Language Recognition**

## Data Preparation

In [2]:
def tfidf_features(X_train, X_test, vectorizer_path):
  """Performs TF-IDF transformation and dumps the model."""
  # Train a vectorizer on X_train data.
  # Transform X_train and X_test data.
  vectorizer = TfidfVectorizer()
  vectorizer.fit(X_train)
  X_train = vectorizer.transform(X_train)
  X_test = vectorizer.transform(X_test)

  # Pickle the trained vectorizer to 'vectorizer_path'
  # Don't forget to open the file in writing bytes mode.
  with open(vectorizer_path, 'wb') as f:
    pickle.dump(vectorizer, f)
    
  return X_train, X_test

In [3]:
sample_size = 200000

dialogue_df = pd.read_csv('data/dialogues.tsv', sep='\t').sample(sample_size, random_state=0)
stackoverflow_df = pd.read_csv('data/tagged_posts.tsv', sep='\t').sample(sample_size, random_state=0)

In [4]:
dialogue_df['text'] = dialogue_df['text'].apply(text_prepare)
stackoverflow_df['title'] = stackoverflow_df['title'].apply(text_prepare)

In [24]:
stackoverflow_df.head()

,post_id,title,tag
2168983,43837842,efficient algorithm compose valid expressions ...,python
1084095,15747223,basic thread program fail clang pass g++,c_cpp
1049020,15189594,link scroll top working,javascript
200466,3273927,possible implement ping windows phone 7,c#
1200249,17684551,glsl normal mapping issue,c_cpp


In [25]:
dialogue_df.head()

,text,tag
82925,donna muffin,dialogue
48774,last night till two oclock hear really got stu...,dialogue
55394,right make appointment see,dialogue
90806,hey thisan interview supposed making love,dialogue
107758,yeah hes friend mine trying help,dialogue


## Intent Recognition

training a model to recognize whether the user has a StackOverflow related question or if he just wants to have a chat

In [26]:
X = np.concatenate([dialogue_df['text'].values, stackoverflow_df['title'].values])
y = ['dialogue'] * dialogue_df.shape[0] + ['stackoverflow'] * stackoverflow_df.shape[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = 0.9, test_size = 0.1 )
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

X_train_tfidf, X_test_tfidf = tfidf_features(X_train, X_test, RESOURCE_PATH['TFIDF_VECTORIZER'])

Train size = 360000, test size = 40000


In [27]:
intent_recognizer = LogisticRegression(penalty='l2', C=10, random_state=0, max_iter=1000)
intent_recognizer.fit(X=X_train_tfidf, y=y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
# Check test accuracy.
y_test_pred = intent_recognizer.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.99055


In [29]:
pickle.dump(intent_recognizer, open(RESOURCE_PATH['INTENT_RECOGNIZER'], 'wb'))

## Programming Language Classification

Trying to determine the tag from the stack overflow post

In [30]:
X = stackoverflow_df['title'].values
y = stackoverflow_df['tag'].values

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

Train size = 160000, test size = 40000


In [32]:
vectorizer = pickle.load(open(RESOURCE_PATH['TFIDF_VECTORIZER'], 'rb'))

X_train_tfidf, X_test_tfidf = vectorizer.transform(X_train), vectorizer.transform(X_test)

In [33]:
tag_classifier = LogisticRegression(penalty='l2', C=5, random_state=0)
tag_classifier = OneVsRestClassifier(tag_classifier)
tag_classifier.fit(X_train_tfidf, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


OneVsRestClassifier(estimator=LogisticRegression(C=5, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=0, solver='lbfgs',
                                                 tol=0.0001, verbose=0,
                                                 warm_start=False),
                    n_jobs=None)

In [34]:
# Check test accuracy.
y_test_pred = tag_classifier.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.7784


In [35]:
pickle.dump(tag_classifier, open(RESOURCE_PATH['TAG_CLASSIFIER'], 'wb'))

# **Ranking questions with embeddings**

In [5]:
import gensim

In [37]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-08-07 14:33:54--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.132.245
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.132.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  16.5MB/s    in 97s     

2020-08-07 14:35:32 (16.1 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [6]:
wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format (fname='GoogleNews-vectors-negative300.bin.gz',binary=True)

In [7]:
# function that enables us to remove words from Word2Vec embeddings

def restrict_w2v(w2v, wanted_word_set):
    new_vectors = []
    new_vocab = {}
    new_index2entity = []

    for i in range(len(w2v.vocab)):
        word = w2v.index2entity[i]
        vec = w2v.vectors[i]
        vocab = w2v.vocab[word]
        if word in wanted_word_set:
            vocab.index = len(new_index2entity)
            new_index2entity.append(word)
            new_vocab[word] = vocab
            new_vectors.append(vec)

    w2v.vocab = new_vocab
    w2v.vectors = new_vectors
    w2v.index2entity = new_index2entity
    w2v.index2word = new_index2entity

In [8]:
# Building Stack Overflow words list
stackoverflow_df['tokens'] = stackoverflow_df['title'].str.split()
setSOwords = set()

for title in stackoverflow_df['tokens']:
  for tk in title:
    setSOwords.add(tk)

49984

In [9]:
restrict_w2v(wv_embeddings, setSOwords)

In [11]:
import csv

with open('data/word_embeddings.tsv', 'w') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    words = wv_embeddings.vocab.keys()
    for word in words:
        vector = wv_embeddings.get_vector(word).tolist()
        row = [word] + vector
        writer.writerow(row)

In [12]:
posts_df = pd.read_csv('data/tagged_posts.tsv', sep='\t')

In [13]:
counts_by_tag = posts_df['tag'].groupby(posts_df['tag']).count()
counts_by_tag

tag
c#            394451
c_cpp         281300
java          383456
javascript    375867
php           321752
python        208607
r              36359
ruby           99930
swift          34809
vb             35044
Name: tag, dtype: int64

In [14]:
os.makedirs(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], exist_ok=True)

for tag, count in counts_by_tag.items():
    tag_posts = posts_df[posts_df['tag'] == tag]
    
    tag_post_ids = tag_posts['post_id'].values
    
    tag_vectors = np.zeros((count, 300), dtype=np.float32)
    for i, title in enumerate(tag_posts['title']):
        tag_vectors[i, :] = question_to_vec(title, wv_embeddings, 300)

    # Dump post ids and vectors to a file.
    filename = os.path.join(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], os.path.normpath('%s.pkl' % tag))
    pickle.dump((tag_post_ids, tag_vectors), open(filename, 'wb'))

In [16]:
dialogue_manager = DialogueManager(RESOURCE_PATH)

Loading resources...
ai.yml Training: [####################] 100%
botprofile.yml Training: [####################] 100%
computers.yml Training: [####################] 100%
conversations.yml Training: [####################] 100%
drugs.yml Training: [####################] 100%
emotion.yml Training: [####################] 100%
food.yml Training: [####################] 100%
gossip.yml Training: [####################] 100%
greetings.yml Training: [####################] 100%
history.yml Training: [####################] 100%
humor.yml Training: [####################] 100%
literature.yml Training: [####################] 100%
money.yml Training: [####################] 100%
movies.yml Training: [####################] 100%
politics.yml Training: [####################] 100%
psychology.yml Training: [####################] 100%
science.yml Training: [####################] 100%
sports.yml Training: [####################] 100%
trivia.yml Training: [####################] 100%


In [17]:
# Testing the algorithm out

questions = [
    "Hey", 
    "How are you doing?", 
    "What's your hobby?", 
    "How to write a loop in python?",
    "How to delete rows in pandas?",
    "python3 re",
    "What is the difference between c and c++",
    "Multithreading in Java",
    "Catch exceptions C++",
    "What is AI?",
]

for question in questions:
    answer = dialogue_manager.generate_answer(question)
    print('Q: %s\nA: %s \n' % (question, answer))

Q: Hey
A: Hi 

Q: How are you doing?
A: I am doing well. 

Q: What's your hobby?
A: Playing Soccer, Painting, Writing are my hobbies.How about you? 

[177234]
[     337      502      683 ... 45810363 45868023 45883655]
Q: How to write a loop in python?
A: I think its about python
This thread might help you: https://stackoverflow.com/questions/33295284 

[202764]
[     337      502      683 ... 45810363 45868023 45883655]
Q: How to delete rows in pandas?
A: I think its about python
This thread might help you: https://stackoverflow.com/questions/40872090 

[0]
[     337      502      683 ... 45810363 45868023 45883655]
Q: python3 re
A: I think its about python
This thread might help you: https://stackoverflow.com/questions/337 

[110890]
[     263      402     1041 ... 45875388 45887857 45898184]
Q: What is the difference between c and c++
A: I think its about c_cpp
This thread might help you: https://stackoverflow.com/questions/9937052 

[61139]
[     564     2092     2158 ... 45832458 